In [1]:
#imports
import pandas as pd
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.metrics import mean_absolute_error

import os
import datetime

import IPython
import IPython.display
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
#import tensorflow as tf

# mpl.rcParams['figure.figsize'] = (8, 6)
# mpl.rcParams['axes.grid'] = False


Plan:
 - Load sets
 - Clean:
    - remove duplicate entries from sets
    - Drop the unimportant coloumns that we found
    - Make querters into hours
    - remove values from train_targets that are nan in both x and y sets
    - make time features
    - drop datetime coloumns
 - Set prep:
    - 75/25 fordeling. 75% av OBSERVED er trening, resterende 25% er validation, 75% av ESTIMATED er trening, resterende 25% er validation
- Concat the different locations into 1 set, where location is a feature as well
   - 1 hot encoding
- Make a time series model

#### Loading all datasets

In [2]:
#A
y_a = pd.read_parquet('../data/A/train_targets.parquet')
X_test_estimated_a = pd.read_parquet('../data/A/X_test_estimated.parquet')
X_train_estimated_a = pd.read_parquet('../data/A/X_train_estimated.parquet')
X_train_observed_a = pd.read_parquet('../data/A/X_train_observed.parquet')

# B
y_b = pd.read_parquet('../data/B/train_targets.parquet')
X_test_estimated_b = pd.read_parquet('../data/B/X_test_estimated.parquet')
X_train_estimated_b = pd.read_parquet('../data/B/X_train_estimated.parquet')
X_train_observed_b = pd.read_parquet('../data/B/X_train_observed.parquet')

# C
y_c = pd.read_parquet('../data/C/train_targets.parquet')
X_test_estimated_c = pd.read_parquet('../data/C/X_test_estimated.parquet')
X_train_estimated_c = pd.read_parquet('../data/C/X_train_estimated.parquet')
X_train_observed_c = pd.read_parquet('../data/C/X_train_observed.parquet')

#### Removing duplicate entries from the sets if any exists

In [3]:
# Function that removes the duplicates if it finds duplicates in the specified coloumn
def remove_duplicates_in_coloumn(df, col):
    duplicate_mask = df[col].duplicated(keep="first")
    if duplicate_mask.any():
        df = df[~duplicate_mask]
    return df

In [4]:
#A
y_a = remove_duplicates_in_coloumn(y_a, "time")
X_test_estimated_a = remove_duplicates_in_coloumn(X_test_estimated_a, "date_forecast")
X_train_estimated_a = remove_duplicates_in_coloumn(X_train_estimated_a, "date_forecast")
X_train_observed_a = remove_duplicates_in_coloumn(X_train_observed_a, "date_forecast")

#B
y_b = remove_duplicates_in_coloumn(y_b, "time")
X_test_estimated_b = remove_duplicates_in_coloumn(X_test_estimated_b, "date_forecast")
X_train_estimated_b = remove_duplicates_in_coloumn(X_train_estimated_b, "date_forecast")
X_train_observed_b = remove_duplicates_in_coloumn(X_train_observed_b, "date_forecast")

#C
y_c = remove_duplicates_in_coloumn(y_c, "time")
X_test_estimated_c = remove_duplicates_in_coloumn(X_test_estimated_c, "date_forecast")
X_train_estimated_c = remove_duplicates_in_coloumn(X_train_estimated_c, "date_forecast")
X_train_observed_c = remove_duplicates_in_coloumn(X_train_observed_c, "date_forecast")


In [5]:
#list of all estimated and observed sets

list_of_all_estimated_and_observed_sets = [X_test_estimated_a, X_train_estimated_a, X_train_observed_a,
                                           X_test_estimated_b, X_train_estimated_b, X_train_observed_b,
                                           X_test_estimated_c, X_train_estimated_c, X_train_observed_c]

#### Dropping some coloumns for the bants

In [6]:
for set in list_of_all_estimated_and_observed_sets: #TODO Not drom any collums
    # set.drop("snow_density:kgm3", axis=1, inplace=True)
    # these 2 had a lot of NaN values
    set.drop("ceiling_height_agl:m", axis=1, inplace=True) 
    set.drop("cloud_base_agl:m", axis=1,inplace=True) # could potentially not drop this, but set all nan values to 0
    set.drop("snow_density:kgm3", axis=1, inplace=True)
    #set.drop("elevation:m", axis=1, inplace=True) 
    #set.drop("precip_5min:mm", axis=1, inplace=True)
    #set.drop("precip_type_5min:idx", axis=1, inplace=True)
    #set.drop("pressure_50m:hPa", axis=1, inplace=True)
    #set.drop("snow_drift:idx", axis=1, inplace=True)
    #set.drop("wind_speed_u_10m:ms", axis=1, inplace=True)
    #set.drop("wind_speed_v_10m:ms", axis=1, inplace=True)
    #set.drop("wind_speed_w_1000hPa:ms", axis=1, inplace=True)

#### Converting degree features to vector

In [7]:
def convert_from_degree_to_ciruclar(df, feature):
    df[feature+'_sin'] = np.sin(np.radians(df[feature]))
    df[feature+'_cos'] = np.cos(np.radians(df[feature]))
    df = df.drop(feature, axis = 1)
    return df

In [8]:
print(X_test_estimated_a.shape)

X_train_estimated_a = convert_from_degree_to_ciruclar(X_train_estimated_a, "sun_azimuth:d")
X_train_observed_a = convert_from_degree_to_ciruclar(X_train_observed_a, "sun_azimuth:d")
X_test_estimated_a = convert_from_degree_to_ciruclar(X_test_estimated_a, "sun_azimuth:d")
X_train_estimated_b = convert_from_degree_to_ciruclar(X_train_estimated_b, "sun_azimuth:d")
X_train_observed_b = convert_from_degree_to_ciruclar(X_train_observed_b, "sun_azimuth:d")
X_test_estimated_b = convert_from_degree_to_ciruclar(X_test_estimated_b, "sun_azimuth:d")
X_train_estimated_c = convert_from_degree_to_ciruclar(X_train_estimated_c, "sun_azimuth:d")
X_train_observed_c = convert_from_degree_to_ciruclar(X_train_observed_c, "sun_azimuth:d")
X_test_estimated_c = convert_from_degree_to_ciruclar(X_test_estimated_c, "sun_azimuth:d")

print(X_test_estimated_a.shape)

# X_train_observed_a.head()

(2880, 44)
(2880, 45)


#### Removing date_calc from all estimated sets


In [9]:
X_test_estimated_a.drop("date_calc", axis=1, inplace=True)
X_train_estimated_a.drop("date_calc", axis=1, inplace=True)
X_test_estimated_b.drop("date_calc", axis=1, inplace=True)
X_train_estimated_b.drop("date_calc", axis=1, inplace=True)
X_test_estimated_c.drop("date_calc", axis=1, inplace=True)
X_train_estimated_c.drop("date_calc", axis=1, inplace=True)

#### Converting every 4 quarters into an whole hour

In [10]:
## It might be better not to resample actually, but just use the actual hours that correspond perfectly

def convert_df_into_hourly(df):
    df.set_index("date_forecast", inplace=True)
    df = df.resample('1H').mean() # TODO change from sum to mean
    df.reset_index(inplace=True)
    return df

In [11]:

print(X_test_estimated_a.shape)

X_train_estimated_a = convert_df_into_hourly(X_train_estimated_a)
X_train_observed_a = convert_df_into_hourly(X_train_observed_a)
X_test_estimated_a = convert_df_into_hourly(X_test_estimated_a)
X_train_estimated_b = convert_df_into_hourly(X_train_estimated_b)
X_train_observed_b = convert_df_into_hourly(X_train_observed_b)
X_test_estimated_b = convert_df_into_hourly(X_test_estimated_b)
X_train_estimated_c = convert_df_into_hourly(X_train_estimated_c)
X_train_observed_c = convert_df_into_hourly(X_train_observed_c)
X_test_estimated_c = convert_df_into_hourly(X_test_estimated_c)

print(X_test_estimated_a.shape)

X_train_estimated_a.head()
## We should probably do it for the test_sets here as well


(2880, 44)
(1536, 44)


,date_forecast,absolute_humidity_2m:gm3,air_density_2m:kgm3,clear_sky_energy_1h:J,clear_sky_rad:W,dew_or_rime:idx,dew_point_2m:K,diffuse_rad:W,diffuse_rad_1h:J,direct_rad:W,...,super_cooled_liquid_water:kgm2,t_1000hPa:K,total_cloud_cover:p,visibility:m,wind_speed_10m:ms,wind_speed_u_10m:ms,wind_speed_v_10m:ms,wind_speed_w_1000hPa:ms,sun_azimuth:d_sin,sun_azimuth:d_cos
0,2022-10-28 22:00:00,8.350,1.23300,0.0,0.0,1.0,281.274994,0.0,0.0,0.0,...,0.00,284.675018,100.0,20712.525391,0.700,-0.325,0.650,0.0,-0.210053,0.973399
1,2022-10-28 23:00:00,8.100,1.23400,0.0,0.0,1.0,280.850006,0.0,0.0,0.0,...,0.00,284.325012,100.0,5624.174805,0.775,0.425,0.525,0.0,0.117007,0.988842
2,2022-10-29 00:00:00,8.150,1.22975,0.0,0.0,1.0,280.924988,0.0,0.0,0.0,...,0.20,283.875000,100.0,3240.425049,1.500,1.050,0.975,0.0,0.408952,0.908216
3,2022-10-29 01:00:00,8.200,1.22850,0.0,0.0,1.0,281.049988,0.0,0.0,0.0,...,0.25,284.350006,100.0,3243.500000,1.575,1.000,1.200,0.0,0.664673,0.742590
4,2022-10-29 02:00:00,8.375,1.22700,0.0,0.0,1.0,281.349976,0.0,0.0,0.0,...,0.25,284.524994,100.0,2528.449951,2.200,1.825,1.175,0.0,0.845577,0.528512


#### Removing all NaN from y_targets
 - Removing all NaN from y_targets and their corresponding dates in the X_train sets, both observed and estimated
 

In [12]:
#Function that returns train_targets, observed and estimated sets left after filtering away NaN
def drop_nan_rows_in_target_and_train(y_df, observed_train_df, estimated_train_df):
    y_df = y_df.dropna(subset=['pv_measurement'])
    valid_dates = y_df['time']
    observed_train_df = observed_train_df[observed_train_df['date_forecast'].isin(valid_dates)]
    estimated_train_df = estimated_train_df[estimated_train_df['date_forecast'].isin(valid_dates)]
    return (y_df, observed_train_df, estimated_train_df)

In [13]:
#doing the NaN filtering
y_a, X_train_observed_a, X_train_estimated_a = drop_nan_rows_in_target_and_train(y_a, X_train_observed_a, X_train_estimated_a)
y_b, X_train_observed_b, X_train_estimated_b = drop_nan_rows_in_target_and_train(y_b, X_train_observed_b, X_train_estimated_b)
y_c, X_train_observed_c, X_train_estimated_c = drop_nan_rows_in_target_and_train(y_c, X_train_observed_c, X_train_estimated_c)

y_c.head()

,time,pv_measurement
5913,2019-09-04 08:00:00,137.2
5914,2019-09-04 09:00:00,0.0
5915,2019-09-04 10:00:00,0.0
5916,2019-09-04 11:00:00,0.0
5917,2019-09-04 12:00:00,0.0


In [14]:
X_train_observed_c.head()

,date_forecast,absolute_humidity_2m:gm3,air_density_2m:kgm3,clear_sky_energy_1h:J,clear_sky_rad:W,dew_or_rime:idx,dew_point_2m:K,diffuse_rad:W,diffuse_rad_1h:J,direct_rad:W,...,super_cooled_liquid_water:kgm2,t_1000hPa:K,total_cloud_cover:p,visibility:m,wind_speed_10m:ms,wind_speed_u_10m:ms,wind_speed_v_10m:ms,wind_speed_w_1000hPa:ms,sun_azimuth:d_sin,sun_azimuth:d_cos
5912,2019-09-04 08:00:00,6.625,1.22075,1320844.375,421.225006,0.0,278.200012,67.974998,233009.921875,338.149994,...,0.0,283.725006,31.199999,49549.449219,1.250,-0.825,0.825,0.0,0.759872,-0.645637
5913,2019-09-04 09:00:00,6.275,1.21425,1681730.500,508.125000,0.0,277.424988,76.625000,260351.078125,406.049988,...,0.0,284.799988,86.250000,52309.976562,1.500,-1.500,-0.200,0.0,0.550184,-0.831061
5914,2019-09-04 10:00:00,5.900,1.20825,1936799.875,561.875000,0.0,276.650024,112.574997,340592.781250,399.700012,...,0.0,285.899994,95.149994,53978.800781,2.000,-2.000,-0.225,0.0,0.279491,-0.956291
5915,2019-09-04 11:00:00,5.875,1.20350,2063526.250,578.025024,0.0,276.649994,195.149994,553897.625000,272.699982,...,0.0,286.924988,100.000000,54510.925781,1.950,-1.950,-0.250,0.0,-0.027237,-0.995790
5916,2019-09-04 12:00:00,6.150,1.20175,2051320.000,555.174988,0.0,277.250000,243.375000,789280.062500,112.349998,...,0.0,287.500000,100.000000,54284.648438,1.525,-1.500,-0.300,0.0,-0.330277,-0.940016


#### Making time features

In [15]:
#Function that creates timefeatures based on the specified coloumn #TODO commented this out
#def create_time_features_based_on_coloun(df, col):   
 #   df['hour'] = df[col].dt.hour
  #  df['dayofmonth'] = df[col].dt.day
   # df['dayofweek'] = df[col].dt.dayofweek
   # df['quarter'] = df[col].dt.quarter
   # df['month'] = df[col].dt.month
   # df['year'] = df[col].dt.year
   # df['dayofyear'] = df[col].dt.dayofyear
   # return df


In [16]:
#Function that creates timefeatures based on the specified coloumn # TODO Made this instead
def create_time_features_based_on_coloun(df, col):   
    df['sin_hour'] = np.sin(((2*np.pi)* df[col].dt.hour) / 23)
    df['cos_hour'] = np.cos(((2*np.pi)* df[col].dt.hour) / 23)
    df['sin_month'] = np.sin(((2*np.pi)* df[col].dt.month) / 12)
    df['cos_month'] = np.cos(((2*np.pi)* df[col].dt.month) / 12)
    #df['hour'] = df[col].dt.hour
    #df['dayofmonth'] = df[col].dt.day
    #df['dayofweek'] = df[col].dt.dayofweek
    #df['quarter'] = df[col].dt.quarter
    #df['month'] = df[col].dt.month
    #df['year'] = df[col].dt.year
    #df['dayofyear'] = df[col].dt.dayofyear
    return df

In [17]:
# adding time features

#A
X_train_estimated_a = create_time_features_based_on_coloun(X_train_estimated_a, "date_forecast")
X_train_observed_a = create_time_features_based_on_coloun(X_train_observed_a, "date_forecast")
X_test_estimated_a = create_time_features_based_on_coloun(X_test_estimated_a, "date_forecast")

#B
X_train_estimated_b = create_time_features_based_on_coloun(X_train_estimated_b, "date_forecast")
X_train_observed_b = create_time_features_based_on_coloun(X_train_observed_b, "date_forecast")
X_test_estimated_b = create_time_features_based_on_coloun(X_test_estimated_b, "date_forecast")

#C
X_train_estimated_c = create_time_features_based_on_coloun(X_train_estimated_c, "date_forecast")
X_train_observed_c = create_time_features_based_on_coloun(X_train_observed_c, "date_forecast")
X_test_estimated_c = create_time_features_based_on_coloun(X_test_estimated_c, "date_forecast")



Just showing the shapes to se if i havent f'ed up

In [18]:
# printing shapes
print("A")
print(X_train_estimated_a.shape)
print(X_train_observed_a.shape)
print(X_train_estimated_a.shape[0]+X_train_observed_a.shape[0])
print(y_a.shape)

print("B")
print(X_train_estimated_b.shape)
print(X_train_observed_b.shape)
print(X_train_estimated_b.shape[0]+X_train_observed_b.shape[0])  ## this one seems to be of by 1??????
print(y_b.shape)

print("C")
print(X_train_estimated_c.shape)
print(X_train_observed_c.shape)
print(X_train_estimated_c.shape[0]+X_train_observed_c.shape[0])
print(y_c.shape)

A
(4418, 48)
(29667, 48)
34085
(34085, 2)
B
(3625, 48)
(29218, 48)
32843
(32844, 2)
C
(2954, 48)
(23141, 48)
26095
(26095, 2)


We can see that for B there is actually on row more in the target set for some reason, finding that row and deleting it:

In [19]:
## Finding the row in y_b that does not have a match

# renaming
date_times_estimated = X_train_estimated_b['date_forecast']
date_times_observed = X_train_observed_b['date_forecast']
result_df = y_b[~y_b['time'].isin(date_times_estimated) & ~y_b['time'].isin(date_times_observed)]
print(result_df)


                 time  pv_measurement
0 2018-12-31 23:00:00             0.0


Okey so let's remove that row and verify that everything is correct:

In [20]:
# removing row
y_b = y_b[~y_b['time'].isin(result_df['time'])]

# checking that the numbers add up again
print("B")
print(X_train_estimated_b.shape)
print(X_train_observed_b.shape)
print(X_train_estimated_b.shape[0]+X_train_observed_b.shape[0])
print(y_b.shape)


B
(3625, 48)
(29218, 48)
32843
(32843, 2)


#### Removing repeated indices


#### Dividing the sets into training and validation

In [21]:
# first renaming the date_forecast columns to time  (DOES NOT WORK)
for df in list_of_all_estimated_and_observed_sets:               
    df.rename(columns={'date_forecast': 'time'}, inplace=True)

#maybe we can drop it here already actually
X_train_observed_a.head()



,date_forecast,absolute_humidity_2m:gm3,air_density_2m:kgm3,clear_sky_energy_1h:J,clear_sky_rad:W,dew_or_rime:idx,dew_point_2m:K,diffuse_rad:W,diffuse_rad_1h:J,direct_rad:W,...,wind_speed_10m:ms,wind_speed_u_10m:ms,wind_speed_v_10m:ms,wind_speed_w_1000hPa:ms,sun_azimuth:d_sin,sun_azimuth:d_cos,sin_hour,cos_hour,sin_month,cos_month
0,2019-06-02 22:00:00,7.700,1.22825,0.000000,0.00,0.0,280.299988,0.000,0.000000,0.00,...,3.600,-3.575,-0.500,0.0,-0.206809,0.976036,-2.697968e-01,0.962917,1.224647e-16,-1.0
1,2019-06-02 23:00:00,7.700,1.22350,0.000000,0.00,0.0,280.299988,0.000,0.000000,0.00,...,3.350,-3.350,0.275,0.0,0.034484,0.997084,-2.449294e-16,1.000000,1.224647e-16,-1.0
2,2019-06-03 00:00:00,7.875,1.21975,0.000000,0.00,0.0,280.649994,0.000,0.000000,0.00,...,3.050,-2.950,0.750,0.0,0.257133,0.964021,0.000000e+00,1.000000,1.224647e-16,-1.0
3,2019-06-03 01:00:00,8.425,1.21800,208.649994,0.75,0.0,281.674988,0.300,526.775024,0.00,...,2.725,-2.600,0.875,0.0,0.478111,0.875816,2.697968e-01,0.962917,1.224647e-16,-1.0
4,2019-06-03 02:00:00,8.950,1.21800,32468.150391,23.10,0.0,282.500000,11.975,22068.949219,0.15,...,2.550,-2.350,0.925,0.0,0.667714,0.741636,5.195840e-01,0.854419,1.224647e-16,-1.0


Training data = first 75% of observed + first 75% of estimated
Validation data = last 25% of observed + last 25% of estimated

In [22]:



# Making training and validation data for A
percent_observed_train_a = 1
percent_estimated_train_a = 1

split_index_obs_a = int(len(X_train_observed_a)*percent_observed_train_a)
X_train_observed_a_first_75 = X_train_observed_a[:split_index_obs_a]
X_train_observed_a_last_25 = X_train_observed_a[split_index_obs_a:]

split_index_est_a = int(len(X_train_estimated_a)*percent_estimated_train_a)
X_train_estimated_a_first_75 = X_train_estimated_a[:split_index_est_a]
X_train_estimated_a_last_25 = X_train_estimated_a[split_index_est_a:]

X_train_a = pd.concat([X_train_observed_a_first_75, X_train_estimated_a_first_75])
y_train_a = y_a[y_a["time"].isin(X_train_a['date_forecast'])]
print(X_train_a.shape, y_train_a.shape)

X_validate_a = pd.concat([X_train_observed_a_last_25, X_train_estimated_a_last_25])
y_validate_a = y_a[y_a["time"].isin(X_validate_a['date_forecast'])]


# making training and validation for B
percent_observed_train_b = 1
percent_estimated_train_b = 1

split_index_obs_b = int(len(X_train_observed_b)*percent_observed_train_b)
X_train_observed_b_first_75 = X_train_observed_b[:split_index_obs_b]
X_train_observed_b_last_25 = X_train_observed_b[split_index_obs_b:]


split_index_est_b = int(len(X_train_estimated_b)*percent_estimated_train_b)
X_train_estimated_b_first_75 = X_train_estimated_b[:split_index_est_b]
X_train_estimated_b_last_25 = X_train_estimated_b[split_index_est_b:]

X_train_b = pd.concat([X_train_observed_b_first_75, X_train_estimated_b_first_75])
y_train_b = y_b[y_b["time"].isin(X_train_b['date_forecast'])]
print(X_train_b.shape, y_train_b.shape)

X_validate_b = pd.concat([X_train_observed_b_last_25, X_train_estimated_b_last_25])
y_validate_b = y_b[y_b["time"].isin(X_validate_b['date_forecast'])]

# making training and validation for C
percent_observed_train_c = 1
percent_estimated_train_c = 1

split_index_obs_c = int(len(X_train_observed_c)*percent_observed_train_c)
X_train_observed_c_first_75 = X_train_observed_c[:split_index_obs_c]
X_train_observed_c_last_25 = X_train_observed_c[split_index_obs_c:]

split_index_est_c = int(len(X_train_estimated_c)*percent_estimated_train_c)
X_train_estimated_c_first_75 = X_train_estimated_c[:split_index_est_c]
X_train_estimated_c_last_25 = X_train_estimated_c[split_index_est_c:]

X_train_c = pd.concat([X_train_observed_c_first_75, X_train_estimated_c_first_75])
y_train_c = y_c[y_c["time"].isin(X_train_c['date_forecast'])]
print(X_train_c.shape, y_train_c.shape)

X_validate_c = pd.concat([X_train_observed_c_last_25, X_train_estimated_c_last_25])
y_validate_c = y_c[y_c["time"].isin(X_validate_c['date_forecast'])]


(34085, 48) (34085, 2)
(32843, 48) (32843, 2)
(26095, 48) (26095, 2)


#### Removing repeated indexes here, think this might be the wrong place to do it, should probably to it for the concat BUT hEY

In [23]:

def find_repeated_indexes(df, column_name, repeat_count=48):
    """
    Find and return the indexes of rows with a specified number of repeated values in a given column.

    Parameters:
    - df: DataFrame to search for repeated rows.
    - column_name: Name of the column to check for repeated values.
    - repeat_count: Number of repeated values required to consider a row as a match.

    Returns:
    - List of indexes for rows with the specified number of repeated values in the given column.
    """
    df = df.reset_index()
    repeated_indexes = []
    temp_repeated_indexes = []
    current_value = None
    count = 0

    for index, row in df.iterrows():

        value = row[column_name]

        if value ==0: #TODO Addedd this check
            continue

        if value == current_value:
            count += 1
            temp_repeated_indexes.append(index)
        else:
            current_value = value
            if count <= repeat_count:
                temp_repeated_indexes = []
                count = 1
            else:
                for i in temp_repeated_indexes:
                    if i not in repeated_indexes:
                        repeated_indexes.append(i)
                temp_repeated_indexes = []
                count = 1
            

    return repeated_indexes



In [24]:
#removing repeated indexes #TODO remove repaeted after 48 hours

#A
print(X_train_a.shape, y_train_a.shape)
repeated_indices = find_repeated_indexes(y_train_a,"pv_measurement", 48)
y_train_a = y_train_a.reset_index()
y_train_a = y_train_a.drop(repeated_indices)
X_train_a = X_train_a[X_train_a["date_forecast"].isin(y_train_a["time"])]
print(X_train_a.shape, y_train_a.shape)

repeated_indices = find_repeated_indexes(y_validate_a,"pv_measurement",48)
y_validate_a = y_validate_a.reset_index()
y_validate_a = y_validate_a.drop(repeated_indices)
X_validate_a = X_validate_a[X_validate_a["date_forecast"].isin(y_validate_a["time"])]

y_train_a.reset_index(drop=True, inplace=True)
X_train_a.reset_index(drop=True, inplace=True)
y_validate_a.reset_index(drop=True, inplace=True)
X_validate_a.reset_index(drop=True, inplace=True)



(34085, 48) (34085, 2)
(34085, 48) (34085, 3)


In [25]:
#B  -  works reeaaaly well for B
print(X_train_b.shape, y_train_b.shape)
repeated_indices = find_repeated_indexes(y_train_b,"pv_measurement", 48)
y_train_b = y_train_b.reset_index()
y_train_b = y_train_b.drop(repeated_indices)
X_train_b = X_train_b[X_train_b["date_forecast"].isin(y_train_b["time"])]
print(X_train_b.shape, y_train_b.shape)

repeated_indices = find_repeated_indexes(y_validate_b,"pv_measurement",48)
y_validate_b = y_validate_b.reset_index()
y_validate_b = y_validate_b.drop(repeated_indices)
X_validate_b = X_validate_b[X_validate_b["date_forecast"].isin(y_validate_b["time"])]

y_train_b.reset_index(drop=True, inplace=True)
X_train_b.reset_index(drop=True, inplace=True)
y_validate_b.reset_index(drop=True, inplace=True)
X_validate_b.reset_index(drop=True, inplace=True)

(32843, 48) (32843, 2)
(29849, 48) (29849, 3)


In [26]:
#C
print(X_train_c.shape, y_train_c.shape)
repeated_indices = find_repeated_indexes(y_train_c,"pv_measurement", 48)
y_train_c = y_train_c.reset_index()
y_train_c = y_train_c.drop(repeated_indices)
X_train_c = X_train_c[X_train_c["date_forecast"].isin(y_train_c["time"])]
print(X_train_c.shape, y_train_c.shape)

repeated_indices = find_repeated_indexes(y_validate_c,"pv_measurement",48)
y_validate_c = y_validate_c.reset_index()
y_validate_c = y_validate_c.drop(repeated_indices)
X_validate_c = X_validate_c[X_validate_c["date_forecast"].isin(y_validate_c["time"])]

y_train_c.reset_index(drop=True, inplace=True)
X_train_c.reset_index(drop=True, inplace=True)
y_validate_c.reset_index(drop=True, inplace=True)
X_validate_c.reset_index(drop=True, inplace=True)

(26095, 48) (26095, 2)
(26095, 48) (26095, 3)


Removing time feature

In [27]:
X_train_a.drop("date_forecast", axis=1, inplace=True)
y_train_a.drop("time", axis=1, inplace=True)
X_validate_a.drop("date_forecast", axis=1, inplace=True)
y_validate_a.drop("time", axis=1, inplace=True)

X_train_b.drop("date_forecast", axis=1, inplace=True)
y_train_b.drop("time", axis=1, inplace=True)
X_validate_b.drop("date_forecast", axis=1, inplace=True)
y_validate_b.drop("time", axis=1, inplace=True)

X_train_c.drop("date_forecast", axis=1, inplace=True)
y_train_c.drop("time", axis=1, inplace=True)
X_validate_c.drop("date_forecast", axis=1, inplace=True)
y_validate_c.drop("time", axis=1, inplace=True)


Adding location features to the sets before merge

In [28]:
X_train_a["location"] =  "A" 
y_train_a["location"] = "A"
X_validate_a["location"] = "A"
y_validate_a["location"] = "A"

X_train_b["location"] = "B"
y_train_b["location"] = "B"
X_validate_b["location"] = "B"
y_validate_b["location"] = "B"

X_train_c["location"] = "C"
y_train_c["location"] = "C"
X_validate_c["location"] = "C"
y_validate_c["location"] = "C"


Merging tranining data:


In [29]:
X_train = pd.concat([X_train_a, X_train_b, X_train_c])
y_train = pd.concat([y_train_a, y_train_b, y_train_c])

X_validate = pd.concat([X_validate_a, X_validate_b, X_validate_c])
y_validate = pd.concat([y_validate_a, y_validate_b, y_validate_c])

print(X_train.shape)
print(y_train.shape)

(90029, 48)
(90029, 3)


One-hot encoding

In [30]:
# one hot encoding and dropping location feature   some fuckery here
# one_hot = pd.get_dummies(X_train["location"]).astype(int)
# X_train = X_train.drop("location", axis=1)
# X_train = pd.merge(X_train, one_hot, left_index=True, right_index=True)


# one_hot = pd.get_dummies(X_validate["location"]).astype(int)
# X_validate = pd.merge(X_validate, one_hot, left_index=True, right_index=True)


# print(X_train.shape)
# print(y_train.shape)

Fixing test set

In [31]:
# load valid dates function
import os


dir_path = os.getcwd()
def load_valid_dates():
    
    test = pd.read_csv(f"{dir_path}/../data/test.csv")

    return test["time"].unique().tolist()

In [32]:
#adding location feature

X_test_estimated_a["location"] = "A"
X_test_estimated_b["location"] = "B"
X_test_estimated_c["location"] = "C"

# concatting:
X_test = pd.concat([X_test_estimated_a, X_test_estimated_b, X_test_estimated_c])
# filtering out invalid dates:
X_test = X_test[X_test["date_forecast"].isin(load_valid_dates())]
# removing forecast coloum
X_test = X_test.drop("date_forecast", axis=1)

One-hot encoding:


In [33]:
#X_train = X_train.reset_index().drop(columns="index")
#one_hot = pd.get_dummies(X_train["location"]).astype(int)
#X_train = X_train.drop("location", axis=1)
#X_train = pd.merge(X_train, one_hot, left_index=True, right_index=True)

#X_validate = X_validate.reset_index().drop(columns="index")
#one_hot = pd.get_dummies(X_validate["location"]).astype(int)
#X_validate = X_validate.drop("location", axis=1)
#X_validate = pd.merge(X_validate, one_hot, left_index=True, right_index=True)

#X_test = X_test.reset_index().drop(columns="index")
#one_hot = pd.get_dummies(X_test["location"]).astype(int)
#X_test = X_test.drop("location", axis=1)
#X_test = pd.merge(X_test, one_hot, left_index=True, right_index=True)

#X_test
# one_hot

#TODO remoce onhot encoding

Normalizing data

In [34]:
columns_to_exclude = ["A", "B", "C", "dew_or_rime:idx", "is_day:idx", "in_shadow:idx", "location"]

columns_to_normalize = [col for col in X_train.columns if col not in columns_to_exclude]

#Mean - std normalization

# X_train_mean = X_train.mean()
# X_train_std = X_train.std()

# X_train = (X_train - X_train_mean) / X_train_std
# X_validate = (X_validate- X_train_mean) / X_train_std
# X_test =  (X_test - X_train_mean) / X_train_std


#Min-max
# Calculate min and max values for scaling
X_min = X_train[columns_to_normalize].min()
X_max = X_train[columns_to_normalize].max()

# Apply min-max scaling to the columns to be normalized
X_train[columns_to_normalize] = (X_train[columns_to_normalize] - X_min) / (X_max - X_min)
X_validate[columns_to_normalize] = (X_validate[columns_to_normalize] - X_min) / (X_max - X_min)
X_test[columns_to_normalize] = (X_test[columns_to_normalize] - X_min) / (X_max - X_min)


Normalizing Y

In [35]:
from sklearn.preprocessing import MinMaxScaler

y_scaler = MinMaxScaler()
print(y_train)


y_train["pv_measurement"] = y_scaler.fit_transform(y_train["pv_measurement"].values.reshape(-1,1))

print(y_train)



       index  pv_measurement location
0          0            0.00        A
1          1            0.00        A
2          2            0.00        A
3          3            0.00        A
4          4           19.36        A
...      ...             ...      ...
26090  32150           50.96        C
26091  32151            2.94        C
26092  32152            0.00        C
26093  32153           -0.00        C
26094  32154           -0.00        C

[90029 rows x 3 columns]
       index  pv_measurement location
0          0        0.000000        A
1          1        0.000000        A
2          2        0.000000        A
3          3        0.000000        A
4          4        0.003377        A
...      ...             ...      ...
26090  32150        0.008888        C
26091  32151        0.000513        C
26092  32152        0.000000        C
26093  32153        0.000000        C
26094  32154        0.000000        C

[90029 rows x 3 columns]


In [36]:
# model = xgb.XGBRegressor(
#     max_depth=7,
#     colsample_bytree=0.8,
#     eta=0.1,
#     n_estimators=90,
#     reg_alpha=0.01,
#     reg_lambda=0.01,
#     enable_categorical=True
# )

# model.fit(X_train, y_train["pv_measurement"])

# X_validate_a_loc = X_validate[X_validate_a["location"]=="A"]
# X_validate_a_loc = X_validate_a_loc.drop("location", axis=1)

# prediction = model.predict(X_validate)

# prediction_a = model.predict(X_validate_a)
# prediction_b = model.predict(X_validate_b)
# prediction_c = model.predict(X_validate_c)

In [37]:
# print(
#     "MAE: ",
#     round(
#         mean_absolute_error(
#             y_true=y_validate["pv_measurement"],
#             y_pred=prediction,
#         ),
#         3,
#     ),
# )

In [38]:
# predicting:
# predict = model.predict(X_test)
# print(predict.head())
# predict = y_scaler.inverse_transform(predict.reshape(-1, 1))
# print(predict.head())

# resultframe = pd.DataFrame(columns = ["id", "prediction"])
# resultframe["prediction"] = predict
# print(resultframe.shape)
# resultframe["id"] = range(len(resultframe))
# resultframe.head()
# # making the csv
# # final = prediction.reset_index().reset_index().rename(columns={"level_0": "id", "y": "prediction"})[["id", "prediction"]]

# resultframe.to_csv("submission_super_all_the_guys_6.csv", index=False)

In [48]:
X_test.head()

,absolute_humidity_2m:gm3,air_density_2m:kgm3,clear_sky_energy_1h:J,clear_sky_rad:W,dew_or_rime:idx,dew_point_2m:K,diffuse_rad:W,diffuse_rad_1h:J,direct_rad:W,direct_rad_1h:J,...,wind_speed_u_10m:ms,wind_speed_v_10m:ms,wind_speed_w_1000hPa:ms,sun_azimuth:d_sin,sun_azimuth:d_cos,sin_hour,cos_hour,sin_month,cos_month,location
0,0.227003,0.488815,0.000000,0.000000,0.0,0.524351,0.000000,0.000000,0.000000,0.000000,...,0.506106,0.682148,0.5,0.637985,0.980437,0.500000,1.000000,0.75,0.066987,A
1,0.224036,0.486330,0.000000,0.000000,0.0,0.520022,0.000000,0.000000,0.000000,0.000000,...,0.496608,0.679245,0.5,0.753681,0.930770,0.635214,0.981372,0.75,0.066987,A
2,0.216617,0.478873,0.000000,0.000000,0.0,0.511363,0.000000,0.000000,0.000000,0.000000,...,0.487110,0.673440,0.5,0.850525,0.856530,0.760399,0.926869,0.75,0.066987,A
3,0.209199,0.473074,0.003385,0.013971,0.0,0.502705,0.028006,0.014057,0.003073,0.001542,...,0.470828,0.670537,0.5,0.924419,0.764388,0.866272,0.840534,0.75,0.066987,A
4,0.201780,0.469760,0.047398,0.091994,0.0,0.496753,0.141598,0.085295,0.037240,0.020301,...,0.462687,0.664731,0.5,0.973761,0.660004,0.944980,0.728769,0.75,0.066987,A


TypeError: 'tuple' object is not callable

In [50]:


X_train = X_train.reset_index().drop(columns="index")
y_train = y_train.reset_index().drop(columns="index")
new_train = pd.merge(X_train, y_train["pv_measurement"], left_index=True, right_index=True)



from catboost import CatBoostRegressor

cat_features = [47]
model = CatBoostRegressor(iterations=1000, depth=9, loss_function="MAE", cat_features=cat_features)


# Fit model
model.fit(X_train, y_train["pv_measurement"])



0:	learn: 0.0502330	total: 1.95s	remaining: 32m 23s
1:	learn: 0.0492023	total: 2.94s	remaining: 24m 27s
2:	learn: 0.0480031	total: 4.15s	remaining: 22m 58s
3:	learn: 0.0469129	total: 4.91s	remaining: 20m 22s
4:	learn: 0.0460152	total: 5.81s	remaining: 19m 16s
5:	learn: 0.0449389	total: 6.51s	remaining: 17m 58s
6:	learn: 0.0438879	total: 7.17s	remaining: 16m 57s
7:	learn: 0.0429560	total: 7.78s	remaining: 16m 5s
8:	learn: 0.0421571	total: 8.37s	remaining: 15m 21s
9:	learn: 0.0412512	total: 8.99s	remaining: 14m 50s
10:	learn: 0.0402339	total: 9.56s	remaining: 14m 19s
11:	learn: 0.0392655	total: 10.1s	remaining: 13m 51s
12:	learn: 0.0383820	total: 10.7s	remaining: 13m 35s
13:	learn: 0.0375352	total: 11.4s	remaining: 13m 25s
14:	learn: 0.0366736	total: 12s	remaining: 13m 6s
15:	learn: 0.0358809	total: 12.5s	remaining: 12m 47s
16:	learn: 0.0351452	total: 13.2s	remaining: 12m 45s
17:	learn: 0.0344032	total: 14s	remaining: 12m 43s
18:	learn: 0.0336703	total: 15s	remaining: 12m 55s
19:	learn: 

In [51]:

# Get predictions
preds = model.predict(X_test)


In [57]:
prediction_df = pd.DataFrame(preds)


In [58]:

prediction_df.head()

,0
0,-1.439017e-06
1,-4.422338e-07
2,8.117553e-06
3,1.241386e-02
4,5.471582e-02


In [59]:
# prediction_df_arr = prediction_df.reshape(-1, 1)
prediction_df_scaled = y_scaler.inverse_transform(prediction_df)

In [63]:
print(prediction_df_scaled)
prediction_df_scaled_df = pd.DataFrame(prediction_df_scaled)
print(prediction_df_scaled_df)

[[-0.00825049]
 [-0.00253551]
 [ 0.04654134]
 ...
 [ 0.65249783]
 [-0.30074861]
 [-0.17035752]]
               0
0      -0.008250
1      -0.002536
2       0.046541
3      71.173898
4     313.708804
...          ...
2155   39.049381
2156   13.459259
2157    0.652498
2158   -0.300749
2159   -0.170358

[2160 rows x 1 columns]


Removing negative values and replacing with 0

In [64]:
resultframe = pd.DataFrame(columns = ["id", "prediction"])
resultframe["prediction"] = prediction_df_scaled_df
resultframe['prediction'] = np.where(resultframe['prediction'] < 0, 0, resultframe['prediction'])
resultframe["id"] = range(len(resultframe))
resultframe.head()

,id,prediction
0,0,-0.008250
1,1,-0.002536
2,2,0.046541
3,3,71.173898
4,4,313.708804


#### Okay lets create a submission

In [65]:

resultframe.to_csv("sumbission_all_the_boys_catboost_and_change_feature_enginnering.csv", index=False)

In [ ]:
# df = pd.read_csv("submission_sivert_super_model_5.csv")

# df['prediction'] = np.where(df['prediction'] < 0, 0, df['prediction'])
# df['prediction'] = np.where(df['prediction'] > 0, df['prediction']+3, 0)
# df.describe()
# df.to_csv("submission_sivert_super_model_5_with_no_negatives_copy_plus_scaled.csv", index=False)
